#PyTorch Intro - Autoróżniczkowanie i Graf Obliczeń - Laboratorium

Do optymalizacji parametrów (wag) sieci neuronowej podczas treningu modelu wykorzystuje się **metodę stochastycznego spadku wzdłuż gradientu**.
Gradient funkcji straty względem parametrów sieci wyznaczany jest algorytmem **propagacji wstecznej** (*back propagation*).

Sieć neuronową możemy potraktować jak złożoną funkcję mapującą wejściowe dane $x \in \mathcal{X}$ (np. obraz czy sekwencję audio) na wyjście $y \in \mathcal{Y}$ parametryzowaną zestawem parametrów (wag) $\theta$.
$$
f_{\theta}( x ) = y
$$
W przypadku $n$-klasowego klasyfikatora wyjściem z sieci jest wektor $y \in \mathbb{R}^n$ nieznormalizowanych wartości, zwanych logitami, z których możemy wyznaczyć rozkład prawdopodobieństwa klas korzystając z funkcji softmax.

W jednym kroku treningu sieci neuronowych wykonujemy:
1. **Przejście w przód** - przetworzenie zestawu wejściowych danych treningowych przez sieć i wyznaczenie wartości wynikowych $y = f_{\theta}(x)$. Następnie wyznaczenie wartości funkcji straty
$\mathcal{L}$
w oparciu o wynikową wartość z sieci i prawdziwą (docelową) wartość.
2. **Przejście w tył** (propagacja wsteczna) - wyznaczenie **gradientu funkcji  straty** $\mathcal{L}$ **względem parametrów sieci** $\theta$.
3. Krok optymalizacji parametrów sieci - zmiana w kierunku przeciwnym do gradientu.

##Przygotowanie środowiska
Upewnij się, że notatnik jest uruchomiony na maszynie z GPU. Jeśli GPU nie jest dostępne zmień typ maszyny (Runtime | Change runtime type) i wybierz T4 GPU.

In [1]:
!nvidia-smi

Fri Mar 21 14:54:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Biblioteka PyTorch (`torch`) jest domyślnie zainstalowana w środowisku COLAB.

In [2]:
import torch
import numpy as np

print(f"Wersja biblioteki PyTorch: {torch.__version__}")

Wersja biblioteki PyTorch: 2.6.0+cu124


Sprawdzenie dostępnego urządzenia GPU.

In [3]:
print(f"Dostępność GPU: {torch.cuda.is_available()}")
print(f"Typ GPU: {torch.cuda.get_device_name(0)}")

Dostępność GPU: True
Typ GPU: Tesla T4


Instalacja pakietu torchviz do wizualizacji grafów obliczeń ([link](https://github.com/szagoruyko/pytorchviz)).

In [ ]:
!pip install -q torchviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.7 MB/s eta 0:00:00


#Automatyczne różniczkowanie (`torch.autograd`)

**Gradient** (lub gradientowe pole wektorowe) funkcji skalarnej wielu zmiennych $
f: \mathbb{R}^D → \mathbb{R}
$ oznaczamyy
$\nabla f$ (czytaj: nabla).
W układzie współrzędnych kartezjańskich gradient jest wektorem, którego składowe są pochodnymi cząstkowymi funkcji $f$:
$$\nabla f=\left[{\frac {\partial f}{\partial x_{1}}},\dots ,{\frac {\partial f}{\partial x_{n}}}\right]$$

Niech $\mathcal{L}: \mathbb{R}^D \rightarrow \mathbb{R}$ będzie pewną funkcją straty określoną dla sieci neuronowej o $D$ parametrach (wagach).
Celem treningu sieci neuronowej jest znalezienie zestawu parametrów $\mathbf{\hat{}} \in \mathbb{R}^D$ minimalizującego wartośc funkcji straty:
$$\mathbf{\hat{w}} = \arg \min_{\textbf{w}} \mathcal{L} \left( \textbf{w} \right)$$
W metodzie **spadku wzdłuż gradientu** zaczynamy od losowo zainicjalizowanych parametrów (wag) sieci $\textbf{w}_0$ a następnie iteracyjnie aktualizujemy parametry sieci w kierunku przeciwnym do wartości gradientu:
$$
\mathbf{w}_{t+1} = \mathbf{w}_{t} - \eta \nabla \mathcal{L} \left( \mathbf{w}_t \right)
$$.





Aby wyznaczyć **gradient funkcji straty względem parametrów sieci**, PyTorch posiada wbudowany mechanizm różniczkowania o nazwie `torch.autograd`. Umożliwia on automatyczne obliczanie gradientu dla dowolnego grafu obliczeniowego.

Obiekty typu Tensor posiadają logiczną flagę `requires_grad`.
Domyślnie flaga `requires_grad` jest ustawiana na `False`.
Po jej włączeniu PyTorch będzie automatycznie budował grafy dla wszystkich obliczeń wykonanych z wykorzystaniem tego tensora aby umożliwić automatyczne wyznaczanie gradientu.
Jeśli jeden z argumentów operacji na tensorach ma ustawioną flagę `requires_grad`, wynik również będzie miał ustawioną tę flagę.

#Zadania do wykonania

##Zadanie 1

Niech $f: \mathbb{R}^2 \rightarrow \mathbb{R}$ będzie funkcją:

$$f(x) = sin(x_1) cos(x_2) + sin(0.5 \cdot x_1) cos(0.5 \cdot x_2)$$.

1.   Napisz kod wyznaczających lokalne minimum funkcji $f$ metodą spadku wzdłuż gradientu dla początkowych wartości argumentów $x_1, x_2$ wylosowanych z zakresu $[0; 10]$. Wyświetl znalezione minimum oraz wartości argumentów funcji.
    *   Wykorzystaj mechanizm autoróżniczkowania do wyznaczenia gradientu funkcji $f$. Pamiętaj, aby włączyć budowanie grafu obliczeń dla tensorów `x1` i `x2`.
    *   Liczbę iteracji i stopę uczenia dobierz eksperymentalnie.
    *   Na końcu każdego kroku optymalizacji wyzeruj wartości gradientów każdego z argumentów (`x.grad.zeros_()`). Domyślnie PyTorch akumuluje wartości gradientu dla wielu wywołań przejścia w tył `backward()`.
2.   Zwizualizuj trajektorie parametrów $(x_1, x_2)$ w kolejnych krokach optymalizacji powtarzając cały proces kilkakrotnie, rozpoczynając od losowo wybranych wartości argumentów, każdy z zakresu $[0; 10]$. Czy za każdym razem osiągane jest to samo lokalne minimum?
3.   (opcjonalnie) Zaimplementuj zwektoryzowaną wersję procedury wykonującej minimalizację wartości funkcji $f$ dla wielu zestawów argumentów wejściowych danych jako macierz (tensor) o wymiarach $(n,2)$.
   *   Zwektoryzowana wersja nie zawiera pętli przechodzącej po każdym z $n$ zestawów argumentów. W jednym kroku optymalizacji aktualizuje wszystkie $n$ zestawów argumentów.
   *   Aby wyznaczyć gradient dla każdego elementu z osobna tensora który nie jest skalarem, np. dla $n$-elementowego wektora `f` zawierającego wyniki obliczeń dla $n$ zestawów argumentów, jako argument metody `backward` podaj tensor jedynek o rozmiarze równym rozmiarowi `f`, np. `f.backward(torch.ones_like(f))`.

Wizualizacja funkcji $f(x)$ z wykorzystaniem biblioteki Plotly.

In [4]:
import numpy as np
import plotly.graph_objects as go

# Utwórz siatkę wartości x i y
x = np.linspace(0, 10, 100)
y = np.linspace(0, 10, 100)
X, Y = np.meshgrid(x, y)
Z = np.sin(X) * np.cos(Y) + np.sin(0.5 * X) * np.cos(0.5 * Y)

fig = go.Figure(data=go.Contour(z=Z, x=x, y=y, colorscale='Viridis'))
fig.update_layout(title="Izolinie funkcji 3D", xaxis_title="X", yaxis_title="Y")
fig.show()

In [8]:
fig = go.Figure(data=[go.Surface(z=Z, x=x, y=y, colorscale='Viridis')])
fig.update_layout(
    scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z")
    )
fig.show()

In [6]:
import matplotlib.pyplot as plt

In [7]:
def f(x):
    x1, x2 = x[:, 0], x[:, 1]
    return torch.sin(x1) * torch.cos(x2) + torch.sin(0.5 * x1) * torch.cos(0.5 * x2)

Zadanie 1.1

Wyznaczenie lokalnego minimum funkcji  f z parametrami:

Learning rate = 0.1 \\
Iterations = 100

In [9]:
learning_rate = 0.1
iterations = 100

# Tworzenie izolinii
x = np.linspace(0, 10, 100)
y = np.linspace(0, 10, 100)
X, Y = np.meshgrid(x, y)
Z = np.sin(X) * np.cos(Y) + np.sin(0.5 * X) * np.cos(0.5 * Y)

fig = go.Figure()
fig.add_trace(go.Contour(z=Z, x=x, y=y, colorscale='Viridis', opacity=0.8))

x_torch = torch.rand(1, 2) * 10
x_torch.requires_grad_(True)

trajectory = []

for _ in range(iterations):
    trajectory.append(x_torch.detach().clone())

    loss = f(x_torch)
    loss.backward(torch.ones_like(loss))

    with torch.no_grad():
        x_torch -= learning_rate * x_torch.grad
        x_torch.grad.zero_()

trajectory = torch.stack(trajectory).squeeze(1).numpy()

min_x1, min_x2 = x_torch.detach().numpy()[0]
min_value = f(x_torch).item()
print(f"Znalezione minimum: f({min_x1:.4f}, {min_x2:.4f}) = {min_value:.4f}")

# Dodanie trajektorii do wykresu
fig.add_trace(go.Scatter(
    x=trajectory[:, 0], y=trajectory[:, 1],
    mode="lines+markers", marker=dict(size=4, color='red'),
    line=dict(width=2, color='red'),
    name=f"Trajektoria {_+1}", showlegend=False if _ > 0 else True
))

fig.update_layout(title="Izolinie funkcji i trajektoria gradient descent",
                  xaxis_title="X1", yaxis_title="X2",
                  legend=dict(x=1.05, y=0.95))

fig.show()


Znalezione minimum: f(4.4113, 6.2831) = -1.7602


Testy dla różnych wartości stopy uczenia i dla różnej liczby iteracji

Learning rate = 0.01, 0.1, 0.5 \\
Iterations = 50, 100, 200

In [13]:
# Tworzenie izolinii
x = np.linspace(0, 10, 100)
y = np.linspace(0, 10, 100)
X, Y = np.meshgrid(x, y)
Z = np.sin(X) * np.cos(Y) + np.sin(0.5 * X) * np.cos(0.5 * Y)

# Lista wartości learning rate i iteracji
learning_rates = [0.01, 0.1, 0.5]
iterations_list = [50, 100, 200]

fig = go.Figure()
fig.add_trace(go.Contour(z=Z, x=x, y=y, colorscale='Viridis', opacity=0.8))

for lr in learning_rates:
    for iters in iterations_list:
        x_torch = torch.rand(1, 2) * 10
        x_torch.requires_grad_(True)

        trajectory = []

        for _ in range(iters):
            trajectory.append(x_torch.detach().clone())

            loss = f(x_torch)
            loss.backward(torch.ones_like(loss))

            with torch.no_grad():
                x_torch -= lr * x_torch.grad
                x_torch.grad.zero_()

        trajectory = torch.stack(trajectory).squeeze(1).numpy()

        min_x1, min_x2 = x_torch.detach().numpy()[0]
        min_value = f(x_torch).item()
        print(f"LR: {lr}, Iterations: {iters} → f({min_x1:.4f}, {min_x2:.4f}) = {min_value:.4f}")

        # Dodanie trajektorii do wykresu
        fig.add_trace(go.Scatter(
            x=trajectory[:, 0], y=trajectory[:, 1],
            mode="lines+markers", marker=dict(size=4),
            line=dict(width=2),
            name=f"LR={lr}, Iter={iters}",
            showlegend=False
        ))

fig.update_layout(title="Izolinie funkcji i trajektorie gradient descent",
                  xaxis_title="X1", yaxis_title="X2",
                  legend=dict(x=1.05, y=0.95))

fig.show()


LR: 0.01, Iterations: 50 → f(1.1428, 9.6265) = -0.8369
LR: 0.01, Iterations: 100 → f(7.7534, 9.7714) = -1.0514
LR: 0.01, Iterations: 200 → f(2.8916, 7.9012) = -0.6965
LR: 0.1, Iterations: 50 → f(7.9285, 9.7934) = -1.0646
LR: 0.1, Iterations: 100 → f(1.6378, 9.0567) = -1.0646
LR: 0.1, Iterations: 200 → f(1.6378, 9.0567) = -1.0646
LR: 0.5, Iterations: 50 → f(7.9210, 2.7735) = -1.0646
LR: 0.5, Iterations: 100 → f(7.9210, 9.7929) = -1.0646
LR: 0.5, Iterations: 200 → f(4.4113, 6.2832) = -1.7602


Zadanie 1.2

Kilkukrotne powtórzenie procesu - 10 epok - z rysunkiem trajektorii

In [14]:
learning_rate = 0.1
iterations = 100
num_experiments = 10

# Tworzenie izolinii
x = np.linspace(0, 10, 100)
y = np.linspace(0, 10, 100)
X, Y = np.meshgrid(x, y)
Z = np.sin(X) * np.cos(Y) + np.sin(0.5 * X) * np.cos(0.5 * Y)

fig = go.Figure()
fig.add_trace(go.Contour(z=Z, x=x, y=y, colorscale='Viridis', opacity=0.8))

for i in range(num_experiments):
    x_torch = torch.rand(1, 2) * 10
    x_torch.requires_grad_(True)

    trajectory = []

    for _ in range(iterations):
        trajectory.append(x_torch.detach().clone())

        loss = f(x_torch)
        loss.backward(torch.ones_like(loss))

        with torch.no_grad():
            x_torch -= learning_rate * x_torch.grad
            x_torch.grad.zero_()

    trajectory = torch.stack(trajectory).squeeze(1).numpy()

    min_x1, min_x2 = x_torch.detach().numpy()[0]
    min_value = f(x_torch).item()
    print(f"Trajektoria {i+1}: f({min_x1:.4f}, {min_x2:.4f}) = {min_value:.4f}")

    # Dodanie trajektorii do wykresu
    fig.add_trace(go.Scatter(
        x=trajectory[:, 0], y=trajectory[:, 1],
        mode="lines+markers", marker=dict(size=4),
        line=dict(width=2),
        name=f"Trajektoria {i+1}",
        showlegend=False
    ))

fig.update_layout(title="Izolinie funkcji i trajektorie gradient descent",
                  xaxis_title="X1", yaxis_title="X2",
                  legend=dict(x=-0.1, y=0.5))

fig.show()


Trajektoria 1: f(4.4113, 6.2832) = -1.7602
Trajektoria 2: f(4.4113, 6.2832) = -1.7602
Trajektoria 3: f(4.4113, 6.2832) = -1.7602
Trajektoria 4: f(4.4113, 6.2832) = -1.7602
Trajektoria 5: f(1.6380, 9.0565) = -1.0646
Trajektoria 6: f(4.4113, 6.2832) = -1.7602
Trajektoria 7: f(7.9210, 2.7736) = -1.0646
Trajektoria 8: f(7.9206, 9.7925) = -1.0646
Trajektoria 9: f(7.9211, 2.7735) = -1.0646
Trajektoria 10: f(1.6387, 3.5106) = -1.0646


Czy zawsze osiągane jest to samo minimum?

Nie, osiągane jest jedno z minimów lokalnych, które znajdują się w bliskim sąsiedztwie punktu zainicjowanego przez x i y.

Zadanie 1.3

Wersja zwektoryzowana

In [20]:
learning_rate = 0.1
iterations = 100
num_points = 10

# Tworzenie izolinii
x = np.linspace(0, 10, 100)
y = np.linspace(0, 10, 100)
X, Y = np.meshgrid(x, y)
Z = np.sin(X) * np.cos(Y) + np.sin(0.5 * X) * np.cos(0.5 * Y)

fig = go.Figure()
fig.add_trace(go.Contour(z=Z, x=x, y=y, colorscale='Viridis', opacity=0.8))

x_torch = torch.rand(num_points, 2) * 10  # (n,2)
x_torch.requires_grad_(True)

trajectories = torch.zeros((iterations, num_points, 2))

for i in range(iterations):
    trajectories[i] = x_torch.detach()

    loss = f(x_torch)
    loss.backward(torch.ones_like(loss))

    with torch.no_grad():
        x_torch -= learning_rate * x_torch.grad
        x_torch.grad.zero_()

trajectories = trajectories.numpy()
for i in range(num_points):
    fig.add_trace(go.Scatter(
        x=trajectories[:, i, 0], y=trajectories[:, i, 1],
        mode="lines+markers", marker=dict(size=4),
        line=dict(width=2),
        name=f"Trajektoria {i+1}", showlegend=False
    ))

final_positions = x_torch.detach().numpy()
final_values = f(x_torch).detach().numpy()
for i in range(num_points):
    print(f"Znalezione minimum {i+1}: f({final_positions[i, 0]:.4f}, {final_positions[i, 1]:.4f}) = {final_values[i]:.4f}")

fig.update_layout(title="Izolinie funkcji i trajektorie gradient descent",
                  xaxis_title="X1", yaxis_title="X2")
fig.show()

Znalezione minimum 1: f(1.6395, 9.0550) = -1.0646
Znalezione minimum 2: f(4.4113, 6.2832) = -1.7602
Znalezione minimum 3: f(4.4113, 6.2832) = -1.7602
Znalezione minimum 4: f(1.6486, 3.5205) = -1.0645
Znalezione minimum 5: f(1.6375, 3.5094) = -1.0646
Znalezione minimum 6: f(4.4113, 6.2832) = -1.7602
Znalezione minimum 7: f(7.9209, 2.7736) = -1.0646
Znalezione minimum 8: f(4.4113, 6.2832) = -1.7602
Znalezione minimum 9: f(1.6376, 3.5095) = -1.0646
Znalezione minimum 10: f(1.6380, 3.5098) = -1.0646


### Gradient Descent - Metoda spadku wzdłuż gradientu

$$f(x) = sin(x_1) cos(x_2) + sin(0.5 \cdot x_1) cos(0.5 \cdot x_2)$$.

In [22]:
def f(x1, x2):
    return np.sin(x1) * np.cos(x2) + np.sin(0.5 * x1) * np.cos(0.5 * x2)

def grad_f(x1, x2):
    df_dx1 = np.cos(x1) * np.cos(x2) + 0.5 * np.cos(0.5 * x1) * np.cos(0.5 * x2)
    df_dx2 = -np.sin(x1) * np.sin(x2) - 0.5 * np.sin(0.5 * x1) * np.sin(0.5 * x2)
    return np.array([df_dx1, df_dx2])

learning_rate = 0.1
iterations = 100

# Tworzenie izolinii
x = np.linspace(0, 10, 100)
y = np.linspace(0, 10, 100)
X, Y = np.meshgrid(x, y)
Z = np.sin(X) * np.cos(Y) + np.sin(0.5 * X) * np.cos(0.5 * Y)

fig = go.Figure()
fig.add_trace(go.Contour(z=Z, x=x, y=y, colorscale='Viridis', opacity=0.8))

x_vec = np.random.rand(2) * 10
trajectory = [x_vec.copy()]

for _ in range(iterations):
    grad = grad_f(x_vec[0], x_vec[1])
    x_vec -= learning_rate * grad
    trajectory.append(x_vec.copy())

trajectory = np.array(trajectory)

# Dodanie trajektorii do wykresu
fig.add_trace(go.Scatter(
    x=trajectory[:, 0], y=trajectory[:, 1],
    mode="lines+markers", marker=dict(size=4),
    line=dict(width=2),
    name=f"Trajektoria {i+1}", showlegend=False
))

min_x1, min_x2 = x_vec
min_value = f(min_x1, min_x2)
print(f"Znalezione minimum: f({min_x1:.4f}, {min_x2:.4f}) = {min_value:.4f}")

fig.update_layout(title="Izolinie funkcji i trajektorie gradient descent",
                  xaxis_title="X1", yaxis_title="X2")

fig.show()


Znalezione minimum: f(7.9208, 2.7737) = -1.0646
